In [82]:
from datasets import load_dataset, Dataset, DatasetDict
import pandas as pd
import datasets
from transformers import (AutoTokenizer, DataCollatorWithPadding, AutoConfig, AutoModel,
                          AutoModelForSequenceClassification, BertConfig, Trainer, BertForSequenceClassification,)
from transformers import TrainingArguments
import evaluate
import torch.nn as nn
import torch
import numpy as np
from utils import PROD_TOK, AUX_TOK

# Load and save data

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [3]:
#Load data
df = pd.read_hdf('human_data.h5', key='df')
df.head()

,Input_seq,Output,Type
0,"[5, 7, 4, 4, 16, 5, 5, 6, 5, 2, 17, 0, 13, 6, ...",[5],Piezas
1,"[1, 6, 8, 4, 1, 9, 11, 7, 2, 5, 8, 10, 1, 12, ...",[5],Productos
2,"[2, 1]",[3],Productos
3,"[9, 5, 6, 9, 11, 1, 2, 6, 8, 12, 0, 6, 7, 10, ...",[7],Productos


In [4]:
#We divide the dataset into products and pieces
df_prod = df.loc[df.Type=='Productos']
df_piez = df.loc[df.Type=='Piezas']

In [6]:
#Save splitted datasets for later use
df_prod.to_hdf('products_human.h5', key='df_prod', index=False)
df_piez.to_hdf('pieces_human.h5', key='df_piez', index=False)

/tmp/ipykernel_65/2260775344.py:2: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block0_values] [items->Index(['Input_seq', 'Output', 'Type'], dtype='object')]

  df_prod.to_hdf('products_human.h5', key='df_prod', index=False)
/tmp/ipykernel_65/2260775344.py:3: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block0_values] [items->Index(['Input_seq', 'Output', 'Type'], dtype='object')]

  df_piez.to_hdf('pieces_human.h5', key='df_piez', index=False)


In [7]:
#Load splitted datasets
df_prod = pd.read_hdf('products_human.h5', key='df_prod')
df_piez = pd.read_hdf('pieces_human.h5', key='df_piez')

# Load and build model

In [146]:
#We define a custom model as our reward model
class RewardModel (torch.nn.Module):
    def __init__(self, model_name, hidden_size,*model_args, **kwargs):
        super().__init__()
        self.base = AutoModelForSequenceClassification.from_pretrained(model_name,*model_args, **kwargs)
        #Last layer must be changed for it to be a regression problem
        self.base.classifier =  torch.nn.Linear(in_features=hidden_size, out_features=1, bias=True)
        self = self.base
        
    #Our custom model should take as input a pair of right/wrong answers with a fixed sequence
    def forward(self, input_ids:torch.tensor, attention_mask:torch.tensor=None,
                token_type_ids:torch.tensor=None, train:bool=False):
        #Whether the model is being trained or tested is important when inferencing
        if train:
            #The input_ids are structured in a way in which the first chunk corresponds to right examples and the second one to wrong ones
            input_right, input_wrong  = input_ids[0], input_ids[1]
            out_right = self.base(input_ids=input_right, attention_mask=attention_mask, token_type_ids=token_type_ids)
            out_wrong = self.base(input_ids=input_wrong, attention_mask=attention_mask, token_type_ids=token_type_ids)
            return out_right, out_wrong
        else:
            return self.base(input_ids=input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids)

In [147]:
#We initialize our custom reward model
rw_model = RewardModel("VCNC/Auto-CNC", hidden_size=768)

In [148]:
#This is an example of how to inference the model with batches of data
input = torch.tensor([[[2, 2, 1, 3, 4, 1], [3, 2, 1, 3, 4, 4]], [[2, 2, 1, 3, 4, 3], [3, 2, 1, 3, 4, 1]]])
token_type = torch.tensor([[0, 0, 0, 0, 0, 1], [0, 0, 0, 0, 0, 1]])
rw_model(input, token_type, train=True)

(SequenceClassifierOutput(loss=None, logits=tensor([[0.2954],
         [0.3358]], grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None),
 SequenceClassifierOutput(loss=None, logits=tensor([[0.4630],
         [0.2988]], grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None))

In [149]:
#We can see that the results do not vary, indicating that our functions is correct
input = torch.tensor([[2, 2, 1, 3, 4, 1], [3, 2, 1, 3, 4, 4]])
token_type = torch.tensor([[0, 0, 0, 0, 0, 1], [0, 0, 0, 0, 0, 1]])
rw_model(input, token_type, train=False)

SequenceClassifierOutput(loss=None, logits=tensor([[0.2954],
        [0.3358]], grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)

# Preprocess data

In [10]:
#In this case we will use the product dataset
data = df_prod.to_numpy()
keys = np.unique(np.array(list(PROD_TOK.values()))) #Get unique tokens only
dataset = np.zeros((len(data)*(len(keys)-1), 3), dtype=object) #The number of pairs is the number of examples at the beginning times 1 minus keys

#Iterate through the dataset to generate sets (correct, wrong, input)
for i in range(len(data)):
    internal_cont = 0
    for j in range(len(keys)):
        if j != data[i, 1][0]:
            dataset[i*(len(keys)-1)+internal_cont, 0] = data[i, 1][0]
            dataset[i*(len(keys)-1)+internal_cont, 1] = j
            dataset[i*(len(keys)-1)+internal_cont, 2] = data[i, 0]
            internal_cont += 1
dataset

array([[5, 0,
        list([1, 6, 8, 4, 1, 9, 11, 7, 2, 5, 8, 10, 1, 12, 7, 12, 6, 6, 11, 8, 3, 12, 7, 9, 11, 9, 5])],
       [5, 1,
        list([1, 6, 8, 4, 1, 9, 11, 7, 2, 5, 8, 10, 1, 12, 7, 12, 6, 6, 11, 8, 3, 12, 7, 9, 11, 9, 5])],
       [5, 2,
        list([1, 6, 8, 4, 1, 9, 11, 7, 2, 5, 8, 10, 1, 12, 7, 12, 6, 6, 11, 8, 3, 12, 7, 9, 11, 9, 5])],
       [5, 3,
        list([1, 6, 8, 4, 1, 9, 11, 7, 2, 5, 8, 10, 1, 12, 7, 12, 6, 6, 11, 8, 3, 12, 7, 9, 11, 9, 5])],
       [5, 4,
        list([1, 6, 8, 4, 1, 9, 11, 7, 2, 5, 8, 10, 1, 12, 7, 12, 6, 6, 11, 8, 3, 12, 7, 9, 11, 9, 5])],
       [5, 6,
        list([1, 6, 8, 4, 1, 9, 11, 7, 2, 5, 8, 10, 1, 12, 7, 12, 6, 6, 11, 8, 3, 12, 7, 9, 11, 9, 5])],
       [5, 7,
        list([1, 6, 8, 4, 1, 9, 11, 7, 2, 5, 8, 10, 1, 12, 7, 12, 6, 6, 11, 8, 3, 12, 7, 9, 11, 9, 5])],
       [5, 8,
        list([1, 6, 8, 4, 1, 9, 11, 7, 2, 5, 8, 10, 1, 12, 7, 12, 6, 6, 11, 8, 3, 12, 7, 9, 11, 9, 5])],
       [5, 9,
        list([1, 6, 8, 4, 1, 9, 11